In [4]:
import pandas as pd
from dash import  html, Input, Output
import plotly.express as px
import dash_bootstrap_components as dbc
from jupyter_dash import JupyterDash


# download dataframe
df = pd.read_csv('dataframe.csv')
df1 = df.copy()
df1.set_index('pais', inplace=True)
df1.drop('code', axis=1, inplace=True)
df2 = df1.pivot_table(columns='pais', sort=False)
tips = df2.columns
df_perc = pd.read_csv('df_perc.csv')

#********************************************************************************************************

fig2 = px.choropleth(df_perc,
                     locations='code',
                     color='variacao',
                     hover_name='pais',
                     color_continuous_scale=px.colors.sequential.PuBu_r,                
                     basemap_visible=False,
                     scope='world',
                     labels={'variacao':'Variação'},
                     template='plotly_white',
                     hover_data={'code': False}
                   )

fig2.update_layout(
    autosize=True,
    coloraxis_colorbar=dict(
        thicknessmode="pixels",
        thickness=10,
        lenmode="pixels",
        len=220,
        yanchor="top",
        y=0.9,
        ticksuffix=" %",
        dtick=5),
    margin_t=80,
    margin_b=0,
    margin_l=10,
    margin_r=0,
    modebar_remove=['zoom', 'pan', 'select', 'zoomIn', 'zoomOut', 'resetScale', 'lasso2d', 'toImage'],
    title_font_size=20,
)

#********************************************************************************************************

filtro_ganho = df_perc[['pais','variacao']].sort_values('variacao', ascending=False).head(30).sort_values('variacao')
filtro_perda = df_perc[['pais','variacao']].sort_values('variacao').head(30).sort_values('variacao')


fig3 = px.bar(filtro_perda, x=filtro_perda.pais, y=filtro_perda.variacao, template='plotly_white', labels={'variacao': 'Variação', 'pais':'País'})
fig3.update_layout(
    yaxis_title=None, xaxis_title=None,
    hoverlabel_bgcolor='#444444',
    modebar_remove=['zoom', 'pan', 'select', 'zoomIn', 'zoomOut', 'resetScale', 'lasso2d', 'toImage'],
    margin_l=0, margin_r=0, margin_b=0, margin_t=0)
fig3.update_traces(
    marker_color='rgb(158,202,225)', 
    marker_line_color='rgb(8,48,107)',
    marker_line_width=1.5,
    opacity=0.6
)
fig3.update_xaxes(side='top')
fig3.update_yaxes(side='right')


fig4 = px.bar(filtro_ganho, x=filtro_ganho.pais, y=filtro_ganho.variacao, template='plotly_white', labels={'variacao': 'Variação', 'pais':'País'})
fig4.update_layout(
    yaxis_title=None, xaxis_title=None,
    hoverlabel_bgcolor='#444444',
    modebar_remove=['zoom', 'pan', 'select', 'zoomIn', 'zoomOut', 'resetScale', 'lasso2d', 'toImage'],
    margin_l=0, margin_r=0, margin_b=0, margin_t=0)
fig4.update_traces(
    marker_color='rgb(158,202,225)', 
    marker_line_color='rgb(8,48,107)',
    marker_line_width=1.5,
    opacity=0.6
)

#********************************************************************************************************

df_tot = pd.read_csv('df_tot.csv', index_col='Unnamed: 0')

fig5 = px.bar(df_tot, x=df_tot.index, y=['Positivos', 'Negativos', 'Neutros'],
              template='simple_white',
              height=300,
              labels={'index':'Ano', 'value': 'Quantidade', 'variable': 'Resultados'},
              color_discrete_sequence=px.colors.sequential.PuBu_r,
              barmode='overlay',
              )
fig5.update_layout(
    modebar_remove=['zoom', 'pan', 'select', 'zoomIn', 'zoomOut', 'resetScale', 'lasso2d', 'toImage'],
    margin_l=0, margin_r=0, margin_b=10, margin_t=10)

#********************************************************************************************************


app = JupyterDash(__name__, external_stylesheets=[dbc.themes.FLATLY])
server = app.server

app.layout = dbc.Container(
    [
        dbc.Row( # inicio da primeira linha (cabeçalho)
            dbc.Col(html.H2(['Variação da Área Florestal (% da área terrestre dos países e territórios) em 30 anos - 1991 a 2020'],
                            style ={'text-align':'center',
                                    'border': 'solid 1px #A2B1C6',
                                    'border-radius': '5px',
                                    'padding': '5px',
                                    'margin-top': '20px',
                                    'margin-left': '73px',
                                    'margin-right': '73px',
                                    'margin-bottom': '10px',
                                    'background-color': '#4682B4',
                                    'color':'#ffffff',
                                    'font-family':'Aller',
                                    'font-size':'26px'}
                           ))
        ),
        
        dbc.Row( # inicio da segunda linha
            dbc.Col(html.Div([
                        html.A(['Fonte de dados:  ']),
                        html.A(href="https://data.worldbank.org/",
                               target="_blank",
                               children=[html.Img(alt="Banco Mundial")]
                              ),
            ],
                style={'text-align':'right',
                       'margin-right': '75px',
                       'font-size': '13px',
                       'line-height': '0px',
                      }
            ))
        ),
        dbc.Row(
            dbc.Col([
                html.P('Definição: Área florestal é a terra sob povoamentos naturais ou plantados com árvores com pelo menos 5 metros no local, produtivas ou não, e exclui povoamentos com árvores em sistemas de produção agrícola (por exemplo, em pomares e sistemas agroflorestais) e árvores em parques e jardins urbanos.')
        ],
                style={
                    'margin-top': '20px',
                    'margin-left': '73px',
                    'margin-right': '73px',
                }
            )
        ),
        
        #html.Br(),
        
        dbc.Row( # inicio da terceira linha
            dbc.Col(html.Div(['Países e territórios',
                              dcc.Dropdown(
                                  id = 'dropdown',
                                  options = df2.columns,
                                  value = 'Brasil'
                              )],
                             style = {
                                 'display': 'inline-block',
                                 'width': '310px',
                                 'margin-left' : '73px',
                                 'margin-bottom':'20px',
                                 'padding-bottom':'0px',
                                 'font-family':'Droid Sans'
                             },
                            ))
        ), # fim da terceira linha
        
        dbc.Row([  # inicio da quarta linha
            
            dbc.Col([  # primeira coluna da quarta linha
                html.H5(['Percentual de área de floresta entre 1991 a 2020'],style={'color':'#394d6a', 'text-align':'center'}),
                dcc.Graph(id='graph', config= {'displaylogo': False},
                    #figure=fig1,
                    style={'margin-left': '73px',
                           'border': 'solid 1px #A2B1C6',
                           'background-color':'white', 
                           'border-radius': '5px',
                           'padding': '5px',
                           #'margin-right': '73px',
                          }),
                #html.Hr(),
            ], width={'size': 6, 'offset': 0, 'order': 1}
            ), # fim da primeira coluna da quarta linha

            dbc.Col([  # segunda coluna da quarta linha
                html.H5(['Percentual de perda / ganho de área florestal em 30 anos'],style={'color':'#394d6a', 'text-align':'center'}),
                dcc.Graph(id='graph_map', config= {'displaylogo': False},
                    figure=fig2,
                    style={'border': 'solid 1px #A2B1C6',
                           'background-color':'white', 
                           'border-radius': '5px',
                           'padding': '5px',
                           'margin-right': '73px',
                             }),
                #html.Hr(),
            ], width={'size': 6, 'offset': 0, 'order': 2}
            ), # fim da segunda coluna da quarta linha
            
        ]), # fim da quarta linha
        
        dbc.Row([ # inicio da quinta linha
            
            dbc.Col([html.Div([ # primeira coluna da quinta linha
                html.H5(['MAIOR PERDA - 30 ANOS'],
                        style={
                            'font-size':'12px',
                            'font-weight': 'bold',
                            'margin-top':'10px',
                            'font-family':'Droid Sans'}),
                html.Abbr(html.H1(['-24,03%'],
                                  style={'font-size':'26px',
                                        'font-family':'Aller'}),
                          title='País com maior perda de área florestal em 30 anos'),
                html.H3(['Nicarágua'],
                        style={'font-size':'18px',
                               'font-weight': 'bold',
                               'font-family':'Droid Sans'})
            ],
                style ={'text-align':'center',
                        'border': 'solid 1px #A2B1C6',
                        'border-radius': '5px',
                        'padding': '5px',
                        'margin-left': '130px',
                        'background-color': '#4682B4',
                        'color':'#ffffff'}
                )]),
            
            dbc.Col([html.Div([ # segunda coluna da quinta linha
                html.H5(['MAIOR GANHO - 30 ANOS'],
                        style={
                            'font-size':'12px',
                            'font-weight': 'bold',
                            'margin-top':'10px',
                            'font-family':'Droid Sans'}),
                html.Abbr(html.H1(['18,62%'],
                                  style={'font-size':'26px',
                                        'font-family':'Aller'}),
                          title='País com maior ganho de área florestal em 30 anos'),
                html.H3(['Porto Rico'],
                        style={'font-size':'18px',
                               'font-weight': 'bold',
                                'font-family':'Droid Sans'})
            ],
                style ={'text-align':'center',
                        'border': 'solid 1px #A2B1C6',
                        'border-radius': '5px',
                        'padding': '5px',
                        #'margin-left': '65px',
                        'background-color': '#4682B4',
                        'color':'#ffffff'}
                )]), 
            
            dbc.Col([html.Div([ # terceira coluna da quinta linha
                html.H5(['MAIOR PERCENTUAL ATUAL'],
                        style={
                            'font-size':'12px',
                            'font-weight': 'bold',
                            'margin-top':'10px',
                            'font-family':'Droid Sans'}),
                html.Abbr(html.H1(['97,41%'],
                                  style={'font-size':'26px',
                                        'font-family':'Aller'}),
                          title='País com maior percentual territorial de área florestal em 2020'),
                html.H3(['Suriname'],
                        style={'font-size':'18px',
                               'font-weight': 'bold',
                                'font-family':'Droid Sans'})
            ],
                style ={'text-align':'center',
                        'border': 'solid 1px #A2B1C6',
                        'border-radius': '5px',
                        'padding': '5px',
                        #'margin-left': '65px',
                        'background-color': '#4682B4',
                        'color':'#ffffff'}
                )]),
            
            dbc.Col([html.Div([ # quarta coluna da quinta linha
                html.H5(['MAIOR PERDA ANUAL'],
                        style={
                            'font-size':'12px',
                            'font-weight': 'bold',
                            'margin-top':'10px',
                            'font-family':'Droid Sans'}),
                html.Abbr(html.H1(['-11,15%'],
                                  style={'font-size':'26px',
                                        'font-family':'Aller'}),
                          title='Território com maior perda de área de floresta em um único ano: 2016'),
                html.H3(['Ilhas Marianas do Norte'],
                        style={'font-size':'18px',
                               'font-weight': 'bold',
                                'font-family':'Droid Sans'})
            ],
                style ={'text-align':'center',
                        'border': 'solid 1px #A2B1C6',
                        'border-radius': '5px',
                        'padding': '5px',
                        #'margin-left': '65px',
                        'background-color': '#4682B4',
                        'color':'#ffffff'}
                )]),
            
            dbc.Col([html.Div([ # quinta coluna da quinta linha
                html.H5(['MAIOR GANHO ANUAL'],
                        style={
                            'font-size':'12px',
                            'font-weight': 'bold',
                            'margin-top':'10px',
                            'font-family':'Droid Sans'}),
                html.Abbr(html.H1(['9,69%'],
                                  style={'font-size':'26px',
                                        'font-family':'Aller'}),
                          title='País com maior ganho de área de floresta em um único ano: 1994'),
                html.H3(['Butão'],
                        style={'font-size':'18px',
                               'font-weight': 'bold',
                                'font-family':'Droid Sans'})
            ],
                style ={'text-align':'center',
                        'border': 'solid 1px #A2B1C6',
                        'border-radius': '5px',
                        'padding': '5px',
                        #'margin-left': '65px',
                        'margin-right': '130px',
                        'background-color': '#4682B4',
                        'color':'#ffffff'}
                )]),
            
        ],
            style={'margin-top':'20px'}
        ),
        
        dbc.Row([ # inicio da sexta linha
            
            dbc.Col( # primeira coluna da sexta linha
                html.H5(['Países e territórios com as maiores perdas em 30 anos'], style={'color':'#394d6a', 'text-align':'center'})),
            dbc.Col( # segunda coluna da sexta linha
                html.H5(['Países e territórios com os maiores ganhos em 30 anos'], style={'color':'#394d6a', 'text-align':'center'}))
        ],
            style={'margin-top':'30px'}
        
        ),
        
        dbc.Row([ # início da sétima linha
            dbc.Col([ # início da primeira coluna da sexta linha
                dcc.Graph(id='graph_1', config= {'displaylogo': False},
                    figure=fig3,
                    style={'margin-left': '50px', 'margin-top': '20px'}),
            ], width={'size': 6, 'offset': 0, 'order': 1}
            ), # fim da primeira coluna da sexta linha
            
            dbc.Col([ # início da primeira coluna da sexta linha
                dcc.Graph(id='graph_2', config= {'displaylogo': False},
                    figure=fig4,
                    style={'margin-right': '50px', 'margin-top': '20px'}),
            ], width={'size': 6, 'offset': 0, 'order': 2}
            ), # fim da primeira coluna da sexta linha
                
        ],
            style={
             'border': 'solid 1px #A2B1C6',
             'background-color':'white',   
             'border-radius': '5px',
             'padding': '5px',
             'margin-left': '73px',
             'margin-right': '73px',
             'margin-bottom':'0px'
         }
        ),
        
        
        dbc.Row([ # início da oitava linha
            dbc.Col([
                html.Div([
                    html.P('Foram analizados 213 países e territórios ao longo de 30 anos, totalizando 6390 resultados, dentre os quais se qualificam nos tipos a seguir:'),
                    html.Ul(html.Li('Resultados Negativos (ano em que houve perda de área florestal em relação ao ano anterior)')),
                    html.Ul(html.Li('Resultados Positivos (ano em que houve ganho de área florestal em relação ao ano anterior)')),
                    html.Ul(html.Li('Resultados Neutros (ano em que não houve variação em relação ao ano anterior (nem ganho, nem perda)')),
                    html.P('A contagem dos resultados pode ser verificada, ano a ano, no gráfico ao lado:')
                ],
                    style={'margin-left':'20px','font-family':'Open Sans','font-size':'17px','color':'#394d6a','margin-top':'20px'})
            ]),
        
           dbc.Col([
               dcc.Graph(id='graph_3', figure=fig5, config= {'displaylogo': False})],               
           ),
        ],
            style={
                   'border': 'solid 1px #A2B1C6',
                   'background-color':'white', 
                   'border-radius': '5px',
                   'padding': '5px',
                   'margin-left': '73px',
                   'margin-right': '73px',
                   'margin-top': '30px',
                   'margin-bottom':'20px'
            }
        ),
        
        html.Br(),
        
        dbc.Row( # início da oitava linha
            dbc.Col(
                html.Div([
                    html.A(['By BessasJr'],
                          )],
                    style={'text-align':'center',
                           'font-size': '13px',
                           'line-height': '0px',
                           'margin-bottom':'20px'
                          }
                ),
            )
        
        )

    ],
    style={'height':'100hv',
           'width':'100hw',
           'background-color':'#F0F8FF',
          },
    fluid=True
)

#********************************************************************************************************

@app.callback(
    Output("graph", "figure"), 
    Input("dropdown", "value"))

def update_bar_chart(value):
    config = {'displaylogo': False}
    fig1 = px.bar(df1.loc[value], x=df1.loc[value].keys(), y=df1.loc[value].values,
                 template='plotly_white',
                 labels={'index': 'Ano', 'y': 'Percentual (%)'},
                )
    fig1.update_yaxes(range=[0, 100])
    fig1.update_layout(title={'text': value, 'y': 0.95, 'x': 0.1},
                      title_font_size=20,
                      title_font_family='Open Sans',
                      title_font_color='#394d6a',
                      modebar_remove=['zoom', 'pan', 'select', 'zoomIn', 'zoomOut', 'resetScale', 'lasso2d', 'toImage'],
                      margin_r=50,
                      hoverlabel_bgcolor='#444444'
                     )
    fig1.update_traces(cliponaxis=False, marker_color='rgb(138,180,225)', marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.6)
    
   
    
    return fig1

#********************************************************************************************************

if __name__ == "__main__":
    app.run_server(debug=True,
                  )

Dash app running on http://127.0.0.1:8050/
